In [ ]:
import os
from ultralytics import YOLO

CURRENT_DIR = os.getcwd()
print("CURRENT_DIR:", CURRENT_DIR)

BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, "..", ".."))
print("BASE_DIR:", BASE_DIR)

DATA_YAML = os.path.join(BASE_DIR, "datasets", "banknote", "data.yaml")
print("DATA_YAML:", DATA_YAML)

model = YOLO("yolov8s.pt")

model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=640,
    batch=8,
    optimizer="AdamW",
    project=os.path.join(BASE_DIR, "models"),
    name="banknote_detector",
    device=0
)


CURRENT_DIR: D:\CurrencyDetectorApp\yolov8_training\notebooks\train
BASE_DIR: D:\CurrencyDetectorApp\yolov8_training
DATA_YAML: D:\CurrencyDetectorApp\yolov8_training\datasets\banknote\data.yaml
Ultralytics 8.3.241  Python-3.10.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\CurrencyDetectorApp\yolov8_training\datasets\banknote\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_